# Notebook Exploration

## Functions

In [22]:
import gensim
import mlflow
import mlflow.sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import spacy

## Split dataset

In [23]:
from sklearn.model_selection import train_test_split

data = pd.read_csv("dataset_cleaned.csv")
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
## Lemmatisation Title et Body

# Charger le modèle de langue spaCy
nlp = spacy.load("en_core_web_sm")

# Lemmatisation pour le champ "title" dans l'ensemble d'entraînement
train_data['title_lemmatized'] = train_data['title'].apply(lambda text: ' '.join([token.lemma_ for token in nlp(text)]))

# Lemmatisation pour le champ "body" dans l'ensemble d'entraînement
train_data['body_lemmatized'] = train_data['body'].apply(lambda text: ' '.join([token.lemma_ for token in nlp(text)]))

# Lemmatisation pour le champ "title" dans l'ensemble de test
test_data['title_lemmatized'] = test_data['title'].apply(lambda text: ' '.join([token.lemma_ for token in nlp(text)]))

# Lemmatisation pour le champ "body" dans l'ensemble de test
test_data['body_lemmatized'] = test_data['body'].apply(lambda text: ' '.join([token.lemma_ for token in nlp(text)]))


## CountVectorizer

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_title = CountVectorizer()
X_title_train = vectorizer_title.fit_transform(train_data['title_lemmatized'])
X_title_test = vectorizer_title.transform(test_data['title_lemmatized'])

vectorizer_body = CountVectorizer()
X_body_train = vectorizer_body.fit_transform(train_data['body_lemmatized'])
X_body_test = vectorizer_body.transform(test_data['body_lemmatized'])

feature_names_title = vectorizer_title.get_feature_names_out()
word_lists_title = [feature_names_title[idx].split() for idx in X_title_train.nonzero()[1]]

## Tf-idf

import spacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

nlp = spacy.load("en_core_web_sm")

tfidf_vectorizer_title = TfidfVectorizer()
X_title_train_tfidf = tfidf_vectorizer_title.fit_transform(train_data['title_lemmatized'])

X_title_test_tfidf = tfidf_vectorizer_title.transform(test_data['title_lemmatized'])

tfidf_vectorizer_body = TfidfVectorizer()
X_body_train_tfidf = tfidf_vectorizer_body.fit_transform(train_data['body_lemmatized'])

X_body_test_tfidf = tfidf_vectorizer_body.transform(test_data['body_lemmatized'])

feature_names_title = tfidf_vectorizer_title.get_feature_names_out()
print("Caractéristiques (mots) apprises par le vectorizer pour 'title':")
print(feature_names_title)

print("Matrice TF-IDF pour 'title' dans l'ensemble d'entraînement:")
print(X_title_train_tfidf.toarray())

print("Matrice TF-IDF pour 'title' dans l'ensemble de test:")
print(X_title_test_tfidf.toarray())

feature_names_body = tfidf_vectorizer_body.get_feature_names_out()
print("Caractéristiques (mots) apprises par le vectorizer pour 'body':")
print(feature_names_body)

print("Matrice TF-IDF pour 'body' dans l'ensemble d'entraînement:")
print(X_body_train_tfidf.toarray())

print("Matrice TF-IDF pour 'body' dans l'ensemble de test:")
print(X_body_test_tfidf.toarray())


## LDA (Latent Dirichlet allocation)

### Préparer LDA sur les titres (pour un modèle simple)

In [27]:
from gensim.models import LdaModel
from gensim.corpora import Dictionary

# Convertir les textes de l'ensemble de test en listes de mots
text_data_test_title = [text.split() for text in test_data['title_lemmatized']]

# dictionnaire à partir des listes de mots de l'ensemble de test
dictionary_test_title = Dictionary(text_data_test_title)

# corpus à partir de l'ensemble de test pour le titre
corpus_test_title = [dictionary_test_title.doc2bow(word_list) for word_list in text_data_test_title]

#  LDA pour le titre sur l'ensemble d'entraînement
lda_model_title = LdaModel(corpus_test_title, num_topics=5, id2word=dictionary_test_title, passes=10, random_state=42)

# Afficher les mots associés à chaque topic pour le titre
print("Mots associés à chaque topic pour le titre:")
for topic_idx, topic in lda_model_title.print_topics():
    print(f"Topic {topic_idx + 1}: {topic}")

# LDA pour transformer les données de test sur l'ensemble de test
lda_transformed_title_test = lda_model_title[corpus_test_title]

# perplexité sur l'ensemble de test
perplexity_test_title = lda_model_title.log_perplexity(corpus_test_title)
print(f"Perplexité pour le modèle LDA (titres) sur l'ensemble de test : {perplexity_test_title}")


Mots associés à chaque topic pour le titre:
Topic 1: 0.027*"a" + 0.027*"?" + 0.027*"to" + 0.027*"*" + 0.018*"(" + 0.018*")" + 0.018*"in" + 0.018*"&" + 0.018*";" + 0.018*"how"
Topic 2: 0.122*"_" + 0.036*"Python" + 0.036*"?" + 0.028*"do" + 0.028*"create" + 0.028*"in" + 0.019*"singleton" + 0.019*")" + 0.019*"(" + 0.019*"a"
Topic 3: 0.040*"to" + 0.040*"in" + 0.032*";" + 0.032*"&" + 0.025*"with" + 0.025*"Pandas" + 0.017*"#" + 0.017*"how" + 0.017*"file" + 0.017*"and"
Topic 4: 0.046*"?" + 0.046*"how" + 0.035*"to" + 0.029*"the" + 0.024*"I" + 0.018*"be" + 0.018*"and" + 0.018*"in" + 0.018*"a" + 0.018*"Python"
Topic 5: 0.045*"?" + 0.031*"a" + 0.023*"why" + 0.023*"be" + 0.023*"use" + 0.023*")" + 0.023*"(" + 0.016*"fast" + 0.016*"run" + 0.016*"Python"
Perplexité pour le modèle LDA (titres) sur l'ensemble de test : -5.584511812417359


### Trouver le nombre de topics avec la Perplexité

In [28]:
from gensim.models import LdaModel
from gensim.corpora import Dictionary
import numpy as np

# Convertir les textes de l'ensemble d'entraînement en listes de mots
text_data_train_title = [text.split() for text in train_data['title_lemmatized']]

# ictionnaire à partir des listes de mots de l'ensemble d'entraînement
dictionary_train_title = Dictionary(text_data_train_title)

#corpus à partir de l'ensemble d'entraînement pour le titre
corpus_train_title = [dictionary_train_title.doc2bow(word_list) for word_list in text_data_train_title]

# Liste des nombres de topics à tester
num_topics_list = [2, 3, 4, 5, 6, 7, 8]

# Calculer la perplexité pour différents nombres de topics
perplexity_scores = []

for num_topics in num_topics_list:
    lda_model_title = LdaModel(corpus_train_title, num_topics=num_topics, id2word=dictionary_train_title, passes=10, random_state=42)
    perplexity_scores.append(lda_model_title.log_perplexity(corpus_train_title))

# nombre de topics avec la perplexité la plus basse
best_num_topics = num_topics_list[np.argmin(perplexity_scores)]

print(f"Le nombre optimal de topics est : {best_num_topics}")


Le nombre optimal de topics est : 7


In [29]:
from sklearn.decomposition import LatentDirichletAllocation

# Définir le nombre de topics
num_topics = best_num_topics

# LDA sur le titre
lda_title = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_title.fit(X_title_train)

# LDA sur le body
lda_body = LatentDirichletAllocation(n_components=num_topics, random_state=42)
lda_body.fit(X_body_train)

M_topics_words = lda_title.components_
M_quest_topics_train = lda_title.transform(X_title_train)

# Afficher les mots associés à chaque topic pour le titre
print("Mots associés à chaque topic pour le titre:")
for topic_idx, topic in enumerate(lda_title.components_):
    top_words_idx = topic.argsort()[:-10 - 1:-1]  # Sélectionner les 10 meilleurs mots pour chaque topic
    top_words = [feature_names_title[i] for i in top_words_idx]
    print(f"Topic {topic_idx + 1}: {', '.join(top_words)}")

# Afficher les mots associés à chaque topic pour le corps
#print("\nMots associés à chaque topic pour le corps:")
#for topic_idx, topic in enumerate(lda_body.components_):
 #   top_words_idx = topic.argsort()[:-10 - 1:-1]  # Sélectionner les 10 meilleurs mots pour #chaque topic
   # top_words = [feature_names_body[i] for i in top_words_idx]
    #print(f"Topic {topic_idx + 1}: {', '.join(top_words)}")

# Transformez les données de test en distributions de topics
X_title_test_topics = lda_title.transform(X_title_test)
X_body_test_topics = lda_body.transform(X_body_test)


Mots associés à chaque topic pour le titre:
Topic 1: in, quot, python, what, 39, be, use, the, and, value
Topic 2: do, vs, in, python, and, list, how, the, be, what
Topic 3: do, how, 39, or, use, of, pip, python, with, date
Topic 4: to, how, be, the, and, find, file, import, between, difference
Topic 5: in, how, 39, python, the, be, do, to, class, type
Topic 6: dataframe, column, of, to, pandas, panda, in, how, row, change
Topic 7: be, to, value, how, why, get, and, division, in, the


In [30]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
import pyLDAvis
import pyLDAvis.lda_model

# Visualisation pour le titre
pyLDAvis.enable_notebook()
vis_title = pyLDAvis.lda_model.prepare(lda_title, X_title_test, vectorizer_title, mds='tsne')
pyLDAvis.display(vis_title)

# Visualisation pour le body
# vis_body = pyLDAvis.sklearn.prepare(lda_body, X_body_test, vectorizer_body, mds='tsne')
# pyLDAvis.display(vis_body)


In [31]:
num_topics = M_topics_words.shape[0]
num_top_words = 10

for topic_idx in range(num_topics):
    top_words_idx = M_topics_words[topic_idx].argsort()[:-num_top_words - 1:-1]
    top_words = [feature_names_title[i] for i in top_words_idx]
    print(f"Topic {topic_idx + 1}: {', '.join(top_words)}")


Topic 1: in, quot, python, what, 39, be, use, the, and, value
Topic 2: do, vs, in, python, and, list, how, the, be, what
Topic 3: do, how, 39, or, use, of, pip, python, with, date
Topic 4: to, how, be, the, and, find, file, import, between, difference
Topic 5: in, how, 39, python, the, be, do, to, class, type
Topic 6: dataframe, column, of, to, pandas, panda, in, how, row, change
Topic 7: be, to, value, how, why, get, and, division, in, the


In [32]:
num_questions = M_quest_topics_train.shape[0]
num_top_topics = 3

for question_idx in range(num_questions):
    top_topics_idx = M_quest_topics_train[question_idx].argsort()[:-num_top_topics - 1:-1]
    print(f"Question {question_idx + 1} - Topics Principaux : {', '.join(map(str, top_topics_idx))}")


Question 1 - Topics Principaux : 4, 0, 3
Question 2 - Topics Principaux : 4, 5, 3
Question 3 - Topics Principaux : 1, 3, 0
Question 4 - Topics Principaux : 3, 6, 5
Question 5 - Topics Principaux : 2, 6, 0
Question 6 - Topics Principaux : 0, 5, 6
Question 7 - Topics Principaux : 5, 6, 3
Question 8 - Topics Principaux : 4, 2, 1
Question 9 - Topics Principaux : 4, 6, 2
Question 10 - Topics Principaux : 1, 0, 4
Question 11 - Topics Principaux : 1, 4, 2
Question 12 - Topics Principaux : 6, 4, 5
Question 13 - Topics Principaux : 4, 0, 3
Question 14 - Topics Principaux : 3, 2, 4
Question 15 - Topics Principaux : 2, 6, 4
Question 16 - Topics Principaux : 4, 0, 3
Question 17 - Topics Principaux : 2, 6, 5
Question 18 - Topics Principaux : 3, 4, 5
Question 19 - Topics Principaux : 3, 5, 0
Question 20 - Topics Principaux : 0, 3, 6
Question 21 - Topics Principaux : 2, 0, 1
Question 22 - Topics Principaux : 3, 0, 1
Question 23 - Topics Principaux : 4, 5, 3
Question 24 - Topics Principaux : 6, 0, 4
Q